# Sparkify Project Workspace

This workspace contains a tiny subset (128MB) of the full dataset available (12GB). This notebook is used to explore a smaller subset with Spark before running on the cloud.

In [1]:
# import libraries
from pyspark.sql import SparkSession, Window
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, concat, count, desc, \
asc, explode, lit, min, max, split, stddev, udf, isnan, when, rank, \
log, sqrt, cbrt, exp
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, \
RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, \
PCA, RegexTokenizer, Tokenizer, StandardScaler, StopWordsRemover, \
StringIndexer, VectorAssembler, MaxAbsScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from time import time
import re
import numpy as np
import scipy
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import random
%matplotlib inline
random.seed(42)

In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

## 1. Load and Clean Dataset

### 1.1. Load data

### 1.2. Check for missing values

## 2. Exploratory Data Analysis (EDA)

Perform EDA using a small subset of the full dataset.

### 2.1. Overview of numerical columns

A quick look at descriptive statistics.

### 2.2. Overview of non-numerical columns

A quick look at the possible categories.

### 2.3. Define churn

### 2.3. Explore data
Analyze the behavior for users who stayed vs users who churned.

## 3. Feature Engineering

In this section, I'm going to build out the features that I think useful to train my model on. First, I'm going to do pilot experiment on a small subset of the full dataset. Then, I will use the code on the full dataset.

## 4. Modeling

## 5. Final Steps